# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-14e02a1fc8-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("F1 pilots explorative search") 

Consider the following exploratory information need:

> explore F1 pilots, their nationality, racing teams, and the country of the parent organization of the team

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P17`     | country       | predicate |
| `wd:Q10497835`| F1 Team       | node |
| `wd:Q10841764`| F1 Driver       | node |
| `wd:Q9671`    | Michael Schumacher      | node |
| `wd:Q169898`  | Scuderia Ferrari         | node |
| `wd:Q27586`   |  Ferrari         | node |
| `wd:Q183`     | Germany       | node |


Also consider

```
?p wdt:P106 wd:Q10841764 .
?p wdt:P27 wd:Q183 .
```

is the BGP to retrieve all **citizens of Germany**

```
?p wdt:P17 wd:Q183 .
?p wdt:P31 wd:Q10497835 .

``` 

is the BGP to retrieve all **F1 teams of Germany**

## Workload Goals

1. Identify the BGP for obtaining the team of a given  F1 driver  and also the F1 competitions in which they participated

2. Identify the BGP that connects a F1 team to a parent company

3. How many F1 drivers and F1 teams exist in each country?

4. Compare number of F1 drivers to their nationality
 
   4.1 Is there some driver that joined a team in their own country?
   
   4.2 Which team employed the larger number of drivers?
   
   4.3 For each country , which are the teams and drivers with the highest number of F1 race participations


In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P106 wd:Q10841764 .
?p wdt:P27 wd:Q183 .

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '58')]


1

In [4]:
# get predicates from a person with profession "F1 Driver"

queryString = """
SELECT DISTINCT ?p ?plabel
WHERE { 

?s wdt:P106 wd:Q10841764 ;
    ?p ?o .

?p <http://schema.org/name> ?plabel .

FILTER (!CONTAINS (?plabel, "ID"))

} 
ORDER BY ?plabel
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1472'), ('plabel', 'Commons Creator page')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('plabel', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P935'), ('plabel', 'Commons gallery')]
[('p', 'http://www.wikidata.org/prop/direct/P7704'), ('plabel', 'Europeana entity')]
[('p', 'http://www.wikidata.org/prop/direct/P4431'), ('plabel', 'Google Doodle')]
[('p', 'http://www.wikidata.org/prop/direct/P213'), ('plabel', 'ISNI')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('plabel', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P5587'), ('plabel', 'Libris-URI')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('plabel', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P2984'), ('plabel', 'Snapchat username')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('plabel', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('plabel', 'U.S.

110

In [5]:
# get predicates from "F1 Driver"

queryString = """
SELECT DISTINCT ?p ?plabel
WHERE { 

wd:Q10841764 ?p ?o .

?p <http://schema.org/name> ?plabel .

FILTER (!CONTAINS (?plabel, "ID"))

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('plabel', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('plabel', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('plabel', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('plabel', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('plabel', 'field of this occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('plabel', "topic's main category")]


6

In [6]:
# get predicates from "F1 Team"

queryString = """
SELECT DISTINCT ?p ?plabel
WHERE { 

wd:Q10497835 ?p ?o .

?p <http://schema.org/name> ?plabel .

FILTER (!CONTAINS (?plabel, "ID"))

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('plabel', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('plabel', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('plabel', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('plabel', "topic's main category")]


4

In [7]:
# get predicates toward "F1 Team"

queryString = """
SELECT DISTINCT ?p ?plabel
WHERE { 

?s ?p wd:Q10497835 .

?p <http://schema.org/name> ?plabel .

FILTER (!CONTAINS (?plabel, "ID"))

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('plabel', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('plabel', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('plabel', 'instance of')]


3

In [8]:
# get predicates and objects from "Michael Schumacher"

queryString = """
SELECT DISTINCT ?p ?plabel (GROUP_CONCAT(?olabel; separator=" | ") as ?objs)
WHERE { 

wd:Q9671 ?p ?o .

?p <http://schema.org/name> ?plabel .
?o <http://schema.org/name> ?olabel .

FILTER (!CONTAINS (?plabel, "ID"))

} 

ORDER BY ?plabel
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('plabel', 'award received'), ('objs', "L'Équipe Champion of Champions | Knight of the Legion of Honour | Commander of the Order of Merit of the Italian Republic | Hall of Fame des deutschen Sports | Laureus World Sports Award for Sportsman of the Year | Silbernes Lorbeerblatt | Princess of Asturias Award for Sports | honorary citizen of Sarajevo | German Sportspersonality of the Year")]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('plabel', 'child'), ('objs', 'Mick Schumacher')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('plabel', 'competition class'), ('objs', '24 Hours of Le Mans | Formula One | Formula Nippon')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('plabel', 'country for sport'), ('objs', 'Germany | Luxembourg')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('plabel', 'country of citizenship'), ('objs', 'Germany')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('plabel', 'des

23

In [9]:
# get subjects and predicates toward "Michael Schumacher"

queryString = """
SELECT DISTINCT (GROUP_CONCAT(?slabel; separator=" | ") as ?subjs) ?p ?plabel 
WHERE { 

?s ?p wd:Q9671 .

?p <http://schema.org/name> ?plabel .
?s <http://schema.org/name> ?slabel .

FILTER (!CONTAINS (?plabel, "ID"))

} 

ORDER BY ?plabel
"""

print("Results")
run_query(queryString)

Results
[('subjs', 'country for sport'), ('p', 'http://www.wikidata.org/prop/direct/P1855'), ('plabel', 'Wikidata property example')]
[('subjs', '1 | Asterix at the Olympic Games | Senna'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('plabel', 'cast member')]
[('subjs', 'Category:Michael Schumacher'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('plabel', "category's main topic")]
[('subjs', 'Mike Shoemaker'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('plabel', 'different from')]
[('subjs', 'list of Formula One Grand Prix wins by Michael Schumacher | Formula One career of Michael Schumacher'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('plabel', 'facet of')]
[('subjs', '1994 Spanish Grand Prix | 2006 Turkish Grand Prix | 2005 San Marino Grand Prix | 2004 European Grand Prix | 2006 European Grand Prix | 1995 German Grand Prix | 2003 Italian Grand Prix | 1998 San Marino Grand Prix | 2004 San Marino Grand Prix | 1997 Monaco Grand Prix | 2004 Monaco Gran

13

In [10]:
# get predicates and objects from "Scuderia Ferrari"

queryString = """
SELECT DISTINCT ?p ?plabel (GROUP_CONCAT(?olabel; separator=" | ") as ?objs)
WHERE { 

wd:Q169898 ?p ?o .

?p <http://schema.org/name> ?plabel .
?o <http://schema.org/name> ?olabel .

FILTER (!CONTAINS (?plabel, "ID"))

} 

ORDER BY ?plabel
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('plabel', 'country'), ('objs', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('plabel', 'instance of'), ('objs', 'Formula One team | auto racing team')]
[('p', 'http://www.wikidata.org/prop/direct/P749'), ('plabel', 'parent organization'), ('objs', 'Ferrari')]
[('p', 'http://www.wikidata.org/prop/direct/P859'), ('plabel', 'sponsor'), ('objs', 'Ray-Ban | Advanced Micro Devices | Lenovo | Banco Santander | Kaspersky | Shell Oil Company | Weichai Holding Group | Hublot | Mahle GmbH')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('plabel', 'sport'), ('objs', 'auto racing')]
[('p', 'http://www.wikidata.org/prop/direct/P355'), ('plabel', 'subsidiary'), ('objs', 'Ferrari Driver Academy')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('plabel', "topic's main category"), ('objs', 'Category:Scuderia Ferrari')]


7

In [11]:
# get subjects and predicates toward "Scuderia Ferrari"

queryString = """
SELECT DISTINCT (GROUP_CONCAT(?slabel; separator=" | ") as ?subjs) ?p ?plabel 
WHERE { 

?s ?p wd:Q169898 .

?p <http://schema.org/name> ?plabel .
?s <http://schema.org/name> ?slabel .

FILTER (!CONTAINS (?plabel, "ID"))

} 

ORDER BY ?plabel
"""

print("Results")
run_query(queryString)

Results
[('subjs', 'Category:Scuderia Ferrari'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('plabel', "category's main topic")]
[('subjs', 'Ben Agathangelou | Ossi Oikarinen | Jean-Claude Migeot | Pat Fry | Mattia Binotto | Loïc Bigois | Jock Clear | Nikolas Tombazis | Maurizio Arrivabene | Diane Holl | Mauro Forghieri | Nigel Stepney'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('plabel', 'employer')]
[('subjs', 'Ferrari in Formula One'), ('p', 'http://www.wikidata.org/prop/direct/P921'), ('plabel', 'main subject')]
[('subjs', 'Ferrari 641 | Ferrari F1/87 | Ferrari SF71H | Ferrari SF70H | Ferrari SF16-H'), ('p', 'http://www.wikidata.org/prop/direct/P176'), ('plabel', 'manufacturer')]
[('subjs', 'Didier Pironi'), ('p', 'http://www.wikidata.org/prop/direct/P463'), ('plabel', 'member of')]
[('subjs', 'Giuseppe Farina | John Surtees | Niki Lauda | Robert Manzon | Eugenio Castellotti | Gilles Villeneuve | Tony Brooks | Achille Varzi | Alain Prost | Sebastian Vettel | Mich

11

In [12]:
# get predicates from "Michael Schumacher" to "Scuderia Ferrari"

queryString = """
SELECT DISTINCT ?p ?plabel 
WHERE { 

wd:Q9671 ?p wd:Q169898 .

?p <http://schema.org/name> ?plabel .

FILTER (!CONTAINS (?plabel, "ID"))

} 

ORDER BY ?plabel
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('plabel', 'member of sports team')]


1

In [13]:
# get predicates from "Scuderia Ferrari" to "Michael Schumacher" 

queryString = """
SELECT DISTINCT ?p ?plabel 
WHERE { 

wd:Q169898 ?p wd:Q9671 .

?p <http://schema.org/name> ?plabel .

FILTER (!CONTAINS (?plabel, "ID"))

} 

ORDER BY ?plabel
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [14]:
# get class of "Scuderia Ferrari"

queryString = """
SELECT DISTINCT ?o ?olabel 
WHERE { 

wd:Q169898 wdt:P31 ?o .

?o <http://schema.org/name> ?olabel .

} 

ORDER BY ?olabel
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q10497835'), ('olabel', 'Formula One team')]
[('o', 'http://www.wikidata.org/entity/Q20074337'), ('olabel', 'auto racing team')]


2

In [15]:
# get predicates and objects from "Formula One team"

queryString = """
SELECT DISTINCT ?p ?plabel (GROUP_CONCAT(?o; separator=" | ") as ?objs) (GROUP_CONCAT(?olabel; separator=" | ") as ?objslbl)
WHERE { 

wd:Q10497835 ?p ?o .

?p <http://schema.org/name> ?plabel .
?o <http://schema.org/name> ?olabel .

} 

ORDER BY ?plabel
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('plabel', 'different from'), ('objs', 'http://www.wikidata.org/entity/Q21748148'), ('objslbl', 'Formula One constructor')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('plabel', 'sport'), ('objs', 'http://www.wikidata.org/entity/Q1968'), ('objslbl', 'Formula One')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('plabel', 'subclass of'), ('objs', 'http://www.wikidata.org/entity/Q20074337'), ('objslbl', 'auto racing team')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('plabel', "topic's main category"), ('objs', 'http://www.wikidata.org/entity/Q6468847'), ('objslbl', 'Category:Formula One entrants')]


4

In [16]:
# get predicates and objects from "Formula One"

queryString = """
SELECT DISTINCT ?p ?plabel (GROUP_CONCAT(?olabel; separator=" | ") as ?objs)
WHERE { 

wd:Q1968 ?p ?o .

?p <http://schema.org/name> ?plabel .
?o <http://schema.org/name> ?olabel .

} 

ORDER BY ?plabel
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('plabel', 'different from'), ('objs', 'F1')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('plabel', 'has part'), ('objs', '1950 Formula One season')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('plabel', 'instance of'), ('objs', 'automobile racing series | world championship')]
[('p', 'http://www.wikidata.org/prop/direct/P2500'), ('plabel', 'league level below'), ('objs', 'Formula Two | Formule 3000')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('plabel', 'operator'), ('objs', 'Formula One Group')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('plabel', 'organizer'), ('objs', "Fédération Internationale de l'Automobile")]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('plabel', 'practiced by'), ('objs', 'Formula One driver')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('plabel', 'sport'), ('objs', 'Formula One racing')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('plabe

11

In [17]:
# get season where "Michael Schumacher" partecipate in

queryString = """
SELECT ?o ?olabel
WHERE { 

wd:Q9671 wdt:P1344 ?o .

?o <http://schema.org/name> ?olabel .

} 
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q69713'), ('olabel', '1994 Formula One season')]
[('o', 'http://www.wikidata.org/entity/Q69722'), ('olabel', '1995 Formula One season')]
[('o', 'http://www.wikidata.org/entity/Q69805'), ('olabel', '2000 Formula One season')]
[('o', 'http://www.wikidata.org/entity/Q69835'), ('olabel', '2001 Formula One season')]
[('o', 'http://www.wikidata.org/entity/Q69844'), ('olabel', '2002 Formula One season')]
[('o', 'http://www.wikidata.org/entity/Q69859'), ('olabel', '2003 Formula One season')]
[('o', 'http://www.wikidata.org/entity/Q69869'), ('olabel', '2004 Formula One season')]


7

In [18]:
# get class of "Formula One season"

queryString = """
SELECT ?o2 ?o2label
WHERE { 

wd:Q9671 wdt:P1344 ?o .
?o wdt:P31 ?o2 .

?o2 <http://schema.org/name> ?o2label .

} 
"""

print("Results")
run_query(queryString)

Results
[('o2', 'http://www.wikidata.org/entity/Q27020041'), ('o2label', 'sports season')]
[('o2', 'http://www.wikidata.org/entity/Q27020041'), ('o2label', 'sports season')]
[('o2', 'http://www.wikidata.org/entity/Q27020041'), ('o2label', 'sports season')]
[('o2', 'http://www.wikidata.org/entity/Q27020041'), ('o2label', 'sports season')]
[('o2', 'http://www.wikidata.org/entity/Q27020041'), ('o2label', 'sports season')]
[('o2', 'http://www.wikidata.org/entity/Q27020041'), ('o2label', 'sports season')]
[('o2', 'http://www.wikidata.org/entity/Q27020041'), ('o2label', 'sports season')]


7

In [19]:
# get predicates and objects from "Formula One season" instances of "Michael Schumacher"

queryString = """
SELECT ?p ?plabel (GROUP_CONCAT(?o2label; separator=" | ") as ?objslbl)
WHERE { 

wd:Q9671 wdt:P1344 ?o .
?o ?p ?o2 .

?p <http://schema.org/name> ?plabel .
?o2 <http://schema.org/name> ?o2label .

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('plabel', 'winner'), ('objslbl', 'Scuderia Ferrari | Scuderia Ferrari | Scuderia Ferrari | Scuderia Ferrari | Scuderia Ferrari | Benetton Formula | Williams Grand Prix Engineering | Michael Schumacher | Michael Schumacher | Michael Schumacher | Michael Schumacher | Michael Schumacher | Michael Schumacher | Michael Schumacher')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('plabel', 'sports season of league or competition'), ('objslbl', 'Formula One | Formula One | Formula One | Formula One | Formula One | Formula One | Formula One')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('plabel', 'organizer'), ('objslbl', "Fédération Internationale de l'Automobile | Fédération Internationale de l'Automobile | Fédération Internationale de l'Automobile | Fédération Internationale de l'Automobile | Fédération Internationale de l'Automobile | Fédération Internationale de l'Automobile | Fédération Internationale de l'Auto

6

In [20]:
# get partecipation in "Formula One" competition of all instances of "F1 Driver"

queryString = """
SELECT DISTINCT ?s ?slabel (GROUP_CONCAT(?olabel; separator=" | ") as ?objslbl)
WHERE { 

?s wdt:P106 wd:Q10841764 ;
    wdt:P1344 ?o .

?s <http://schema.org/name> ?slabel .
?o <http://schema.org/name> ?olabel .

FILTER (CONTAINS(?olabel, "Formula One"))

} 
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q10490'), ('slabel', 'Ayrton Senna'), ('objslbl', '1989 Formula One season | 1988 Formula One season | 1985 Formula One season | 1987 Formula One season | 1990 Formula One season | 1991 Formula One season | 1994 Formula One season | 1986 Formula One season | 1992 Formula One season | 1984 Formula One season | 1993 Formula One season')]
[('s', 'http://www.wikidata.org/entity/Q171381'), ('slabel', 'Keke Rosberg'), ('objslbl', 'Formula One')]
[('s', 'http://www.wikidata.org/entity/Q172285'), ('slabel', 'Eppie Wietzes'), ('objslbl', '1974 Formula One season | 1967 Formula One season')]
[('s', 'http://www.wikidata.org/entity/Q173238'), ('slabel', 'Carlos Reutemann'), ('objslbl', 'Formula One')]
[('s', 'http://www.wikidata.org/entity/Q16149172'), ('slabel', 'Alexander Albon'), ('objslbl', 'Formula One | 2019 Formula One World Championship')]
[('s', 'http://www.wikidata.org/entity/Q17319645'), ('slabel', 'George Russell'), ('objslbl', 'Formula On

12

In [21]:
# GOAL (1)
# get "F2 Driver" instances whith their partecipation in "Formula One" competition and their teams
# Replace ?s with the code of a specific driver to get related data

queryString = """
SELECT DISTINCT ?s ?slabel (GROUP_CONCAT(DISTINCT ?olabel; separator=" | ") as ?objslbl) (GROUP_CONCAT(DISTINCT ?o2label; separator=" | ") as ?objs2lbl)
WHERE { 

?s wdt:P106 wd:Q10841764 ;
    wdt:P1344 ?o ;
    wdt:P54 ?o2 .

?s <http://schema.org/name> ?slabel .
?o <http://schema.org/name> ?olabel .
?o2 <http://schema.org/name> ?o2label .

FILTER (CONTAINS(?olabel, "Formula One"))

} 
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q10490'), ('slabel', 'Ayrton Senna'), ('objslbl', '1984 Formula One season | 1985 Formula One season | 1986 Formula One season | 1987 Formula One season | 1988 Formula One season | 1989 Formula One season | 1990 Formula One season | 1991 Formula One season | 1992 Formula One season | 1993 Formula One season | 1994 Formula One season'), ('objs2lbl', 'McLaren | Team Lotus | Toleman | Williams Grand Prix Engineering')]
[('s', 'http://www.wikidata.org/entity/Q171381'), ('slabel', 'Keke Rosberg'), ('objslbl', 'Formula One'), ('objs2lbl', 'ATS F1 | Fittipaldi Automotive | McLaren | Theodore Racing | Walter Wolf Racing | Williams Grand Prix Engineering')]
[('s', 'http://www.wikidata.org/entity/Q172285'), ('slabel', 'Eppie Wietzes'), ('objslbl', '1967 Formula One season | 1974 Formula One season'), ('objs2lbl', 'Brabham | Team Lotus')]
[('s', 'http://www.wikidata.org/entity/Q16149172'), ('slabel', 'Alexander Albon'), ('objslbl', '2019 Formula One 

11

In [22]:
# GOAL (2)
# get all "Formula One team" instances and their parent organization

queryString = """
SELECT ?s ?slabel ?o ?olabel
WHERE { 

?s wdt:P31 wd:Q10497835 ;
    wdt:P749 ?o .

?s <http://schema.org/name> ?slabel .
?o <http://schema.org/name> ?olabel .


} 
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q172030'), ('slabel', 'McLaren'), ('o', 'http://www.wikidata.org/entity/Q173146'), ('olabel', 'McLaren Technology Group')]
[('s', 'http://www.wikidata.org/entity/Q2704273'), ('slabel', 'Aston Martin F1 Team'), ('o', 'http://www.wikidata.org/entity/Q27074'), ('olabel', 'Aston Martin Lagonda')]
[('s', 'http://www.wikidata.org/entity/Q169898'), ('slabel', 'Scuderia Ferrari'), ('o', 'http://www.wikidata.org/entity/Q27586'), ('olabel', 'Ferrari')]
[('s', 'http://www.wikidata.org/entity/Q173314'), ('slabel', 'Enzo Coloni Racing Car Systems'), ('o', 'http://www.wikidata.org/entity/Q1631454'), ('olabel', 'Scuderia Coloni')]


4

In [23]:
# get partecipation in "Formula One" competition of all instances of "F1 Driver" and the class of competitions instances

queryString = """
SELECT DISTINCT ?s ?slabel (GROUP_CONCAT(?olabel; separator=" | ") as ?objslbl) ?o2 ?o2label
WHERE { 

?s wdt:P106 wd:Q10841764 ;
    wdt:P1344 ?o .
    
?o wdt:P31 ?o2 .

?s <http://schema.org/name> ?slabel .
?o <http://schema.org/name> ?olabel .
?o2 <http://schema.org/name> ?o2label .

FILTER (CONTAINS(?olabel, "Formula One"))

} 
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q10490'), ('slabel', 'Ayrton Senna'), ('objslbl', '1989 Formula One season | 1988 Formula One season | 1985 Formula One season | 1987 Formula One season | 1990 Formula One season | 1991 Formula One season | 1994 Formula One season | 1986 Formula One season | 1992 Formula One season | 1984 Formula One season | 1993 Formula One season'), ('o2', 'http://www.wikidata.org/entity/Q27020041'), ('o2label', 'sports season')]
[('s', 'http://www.wikidata.org/entity/Q82742'), ('slabel', 'Romain Grosjean'), ('objslbl', 'Formula One'), ('o2', 'http://www.wikidata.org/entity/Q60147807'), ('o2label', 'automobile racing series')]
[('s', 'http://www.wikidata.org/entity/Q16149172'), ('slabel', 'Alexander Albon'), ('objslbl', '2019 Formula One World Championship'), ('o2', 'http://www.wikidata.org/entity/Q27020041'), ('o2label', 'sports season')]
[('s', 'http://www.wikidata.org/entity/Q172832'), ('slabel', 'David Coulthard'), ('objslbl', 'Formula One'), ('o2',

20

In [24]:
# GOAL (3)
# get counts of F1 drivers and teams for each country

queryString = """
SELECT ?clabel (COUNT(DISTINCT ?team) as ?teamcount) (COUNT(DISTINCT ?driver) as ?drivercount)
WHERE { 
{
    ?team wdt:P31 wd:Q10497835 ;
        wdt:P17 ?country .
    ?country <http://schema.org/name> ?clabel .
}
UNION
{
    ?driver wdt:P106 wd:Q10841764 ;
        wdt:P1344 ?o ;
        wdt:P27 ?country .
    ?o <http://schema.org/name> ?olabel .
    ?country <http://schema.org/name> ?clabel .
    
    FILTER (CONTAINS(?olabel, "Formula One"))
}

} 
GROUP BY ?clabel
"""

print("Results")
run_query(queryString)

Results
[('clabel', 'Brazil'), ('teamcount', '0'), ('drivercount', '1')]
[('clabel', 'Malaysia'), ('teamcount', '1'), ('drivercount', '0')]
[('clabel', 'Argentina'), ('teamcount', '0'), ('drivercount', '1')]
[('clabel', 'United Kingdom'), ('teamcount', '26'), ('drivercount', '3')]
[('clabel', 'Canada'), ('teamcount', '0'), ('drivercount', '1')]
[('clabel', 'Spain'), ('teamcount', '1'), ('drivercount', '2')]
[('clabel', 'Netherlands'), ('teamcount', '2'), ('drivercount', '0')]
[('clabel', 'Australia'), ('teamcount', '1'), ('drivercount', '0')]
[('clabel', 'Japan'), ('teamcount', '2'), ('drivercount', '0')]
[('clabel', 'Russia'), ('teamcount', '2'), ('drivercount', '0')]
[('clabel', 'Finland'), ('teamcount', '0'), ('drivercount', '1')]
[('clabel', 'France'), ('teamcount', '6'), ('drivercount', '1')]
[('clabel', 'Thailand'), ('teamcount', '0'), ('drivercount', '1')]
[('clabel', 'Germany'), ('teamcount', '2'), ('drivercount', '1')]
[('clabel', 'Italy'), ('teamcount', '16'), ('drivercount',

16

In [25]:
# QUERY (4.1)
# get drivers which join a team situated in their country of citizienship
# Answer: TRUE

queryString = """
SELECT ?driver ?dlabel ?dclabel ?team ?tlabel ?tclabel 
WHERE { 

?driver wdt:P106 wd:Q10841764 ;
        wdt:P1344 ?o ;
        wdt:P27 ?dcountry ;
        wdt:P54 ?team .
        
?driver <http://schema.org/name> ?dlabel .
?dcountry <http://schema.org/name> ?dclabel .
?o <http://schema.org/name> ?olabel .
        
?team wdt:P31 wd:Q10497835 ;
        wdt:P17 ?tcountry .
        
?team <http://schema.org/name> ?tlabel .
?tcountry <http://schema.org/name> ?tclabel .
    
FILTER (CONTAINS(?olabel, "Formula One"))
FILTER (?dcountry = ?tcountry)

} 
"""

print("Results")
run_query(queryString)

Results
[('driver', 'http://www.wikidata.org/entity/Q172832'), ('dlabel', 'David Coulthard'), ('dclabel', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q172030'), ('tlabel', 'McLaren'), ('tclabel', 'United Kingdom')]
[('driver', 'http://www.wikidata.org/entity/Q172832'), ('dlabel', 'David Coulthard'), ('dclabel', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q173663'), ('tlabel', 'Red Bull Racing'), ('tclabel', 'United Kingdom')]
[('driver', 'http://www.wikidata.org/entity/Q172832'), ('dlabel', 'David Coulthard'), ('dclabel', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q171337'), ('tlabel', 'Williams Grand Prix Engineering'), ('tclabel', 'United Kingdom')]
[('driver', 'http://www.wikidata.org/entity/Q16149172'), ('dlabel', 'Alexander Albon'), ('dclabel', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q173663'), ('tlabel', 'Red Bull Racing'), ('tclabel', 'United Kingdom')]
[('driver', 'http://www.wikidata.org/entity/Q17319645'), (

7

In [26]:
# GOAL (4.2)
# get the count of the drivers for each team
# Answer: McLaren with 28 drivers

queryString = """
SELECT ?team ?tlabel (COUNT(?driver) as ?numberofdrivers)
WHERE { 

?driver wdt:P106 wd:Q10841764 ;
        wdt:P54 ?team ;
        wdt:P1344 ?o .

?team wdt:P31 wd:Q10497835 .

?driver <http://schema.org/name> ?dlabel .
?team <http://schema.org/name> ?tlabel .
?o <http://schema.org/name> ?olabel .

FILTER (CONTAINS(?olabel, "Formula One"))

} 
GROUP BY ?team ?tlabel
ORDER BY DESC (?numberofdrivers)
"""

print("Results")
run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q172030'), ('tlabel', 'McLaren'), ('numberofdrivers', '28')]
[('team', 'http://www.wikidata.org/entity/Q169898'), ('tlabel', 'Scuderia Ferrari'), ('numberofdrivers', '22')]
[('team', 'http://www.wikidata.org/entity/Q142381'), ('tlabel', 'Minardi'), ('numberofdrivers', '17')]
[('team', 'http://www.wikidata.org/entity/Q172722'), ('tlabel', 'Renault F1 Team'), ('numberofdrivers', '15')]
[('team', 'http://www.wikidata.org/entity/Q98930155'), ('tlabel', 'Alpine F1 Team'), ('numberofdrivers', '15')]
[('team', 'http://www.wikidata.org/entity/Q171337'), ('tlabel', 'Williams Grand Prix Engineering'), ('numberofdrivers', '14')]
[('team', 'http://www.wikidata.org/entity/Q171367'), ('tlabel', 'Jordan Grand Prix'), ('numberofdrivers', '7')]
[('team', 'http://www.wikidata.org/entity/Q172721'), ('tlabel', 'Mercedes'), ('numberofdrivers', '7')]
[('team', 'http://www.wikidata.org/entity/Q171329'), ('tlabel', 'Benetton Formula'), ('numberofdrivers', '7')

14

In [27]:
# get predicates from "sport season"

queryString = """
SELECT DISTINCT ?p ?plabel (GROUP_CONCAT(?o2label; separator=" | ") as ?objslbl) 
WHERE { 

?driver wdt:P106 wd:Q10841764 ;
        wdt:P1344 ?o .
        
?o ?p ?o2 .

?p <http://schema.org/name> ?plabel .
?o <http://schema.org/name> ?olabel .
?o2 <http://schema.org/name> ?o2label .

FILTER (CONTAINS(?olabel, "Formula One"))

} 
GROUP BY ?p ?plabel
ORDER BY (?p)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('plabel', "topic's main Wikimedia portal"), ('objslbl', 'Portal:Formula One | Portal:Formula One | Portal:Formula One | Portal:Formula One | Portal:Formula One | Portal:Formula One | Portal:Formula One')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('plabel', 'winner'), ('objslbl', 'Ayrton Senna | Ayrton Senna | Ayrton Senna | Nelson Piquet | Niki Lauda | Alain Prost | Alain Prost | Alain Prost | Alain Prost | Williams Grand Prix Engineering | Williams Grand Prix Engineering | Williams Grand Prix Engineering | Williams Grand Prix Engineering | Williams Grand Prix Engineering | McLaren | McLaren | McLaren | McLaren | McLaren | McLaren | Michael Schumacher | Nigel Mansell | Alain Prost | McLaren | Ayrton Senna | McLaren | Scuderia Ferrari | Scuderia Ferrari | Scuderia Ferrari | Scuderia Ferrari | Scuderia Ferrari | Benetton Formula | Williams Grand Prix Engineering | Michael Schumacher | Michael Schumacher | Michael S

17

In [28]:
# get "Grand Prix" from each Formula 1 season

queryString = """
SELECT DISTINCT ?o ?olabel (GROUP_CONCAT(?o2label; separator=" | ") as ?objslbl) 
WHERE { 

?driver wdt:P106 wd:Q10841764 ;
        wdt:P1344 ?o .
        
?o wdt:P527 ?o2 .

?o <http://schema.org/name> ?olabel .
?o2 <http://schema.org/name> ?o2label .

FILTER (CONTAINS(?olabel, "Formula One"))
FILTER (CONTAINS(?o2label, "Prix"))

} 
GROUP BY ?o ?olabel
ORDER BY (?olabel)
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q69229'), ('olabel', '1967 Formula One season'), ('objslbl', '1967 Belgian Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q69343'), ('olabel', '1974 Formula One season'), ('objslbl', '1974 Brazilian Grand Prix | 1974 Belgian Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q69576'), ('olabel', '1984 Formula One season'), ('objslbl', '1984 Belgian Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q69603'), ('olabel', '1985 Formula One season'), ('objslbl', '1985 Australian Grand Prix | 1985 Belgian Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q69614'), ('olabel', '1986 Formula One season'), ('objslbl', '1986 Australian Grand Prix | 1986 Belgian Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q69623'), ('olabel', '1987 Formula One season'), ('objslbl', '1987 Australian Grand Prix | 1987 Belgian Grand Prix')]
[('o', 'http://www.wikidata.org/entity/Q69633'), ('olabel', '1988 Formula One season'), ('objslbl', '1988 Bel

17

In [29]:
# get partecipating team for each season and related number of prix

queryString = """
SELECT ?country ?countrylbl ?team ?teamlbl ?clabel (COUNT(?prix) as ?tnumprix)
WHERE {

?team wdt:P31 wd:Q10497835 ;
    wdt:P17 ?country .

?champs wdt:P1923 ?team .

?champs wdt:P527 ?prix .

?champs <http://schema.org/name> ?clabel .
?prix <http://schema.org/name> ?plabel .

?country <http://schema.org/name> ?countrylbl .
?team <http://schema.org/name> ?teamlbl .

FILTER (CONTAINS(?clabel, "Formula One"))
FILTER (CONTAINS(?plabel, "Prix"))

}
GROUP BY ?country ?countrylbl ?team ?teamlbl ?clabel
ORDER BY ?country ?clabel
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('countrylbl', 'France'), ('team', 'http://www.wikidata.org/entity/Q172722'), ('teamlbl', 'Renault F1 Team'), ('clabel', '2016 Formula One World Championship'), ('tnumprix', '21')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countrylbl', 'France'), ('team', 'http://www.wikidata.org/entity/Q172722'), ('teamlbl', 'Renault F1 Team'), ('clabel', '2017 Formula One World Championship'), ('tnumprix', '20')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countrylbl', 'France'), ('team', 'http://www.wikidata.org/entity/Q172722'), ('teamlbl', 'Renault F1 Team'), ('clabel', '2018 Formula One World Championship'), ('tnumprix', '18')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q16300222'), ('teamlbl', 'Haas F1 Team'), ('clabel', '2016 Formula One World Championship'), ('tnumprix', '21')]
[('country', 'http://www.wikidata.org/entity/Q145

33

In [30]:
# get partecipating driver for each season and related number of prix

queryString = """
SELECT ?country ?countrylbl ?driver ?driverlbl (SUM(?dnumprix) as ?dtotnumprix)
    WHERE {
        SELECT ?country ?countrylbl ?driver ?driverlbl (COUNT(?prix) as ?dnumprix)
        WHERE {

        ?driver wdt:P106 wd:Q10841764 ;
                wdt:P1344 ?champs ;
                wdt:P27 ?country .

        ?champs wdt:P527 ?prix .

        ?champs <http://schema.org/name> ?clabel .
        ?prix <http://schema.org/name> ?plabel .

        ?country <http://schema.org/name> ?countrylbl .
        ?driver <http://schema.org/name> ?driverlbl .

        FILTER (CONTAINS(?clabel, "Formula One"))

        }
        GROUP BY ?country ?countrylbl ?driver ?driverlbl
        ORDER BY ?country
    }
    GROUP BY ?country ?countrylbl ?driver ?driverlbl
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('driver', 'http://www.wikidata.org/entity/Q16149172'), ('driverlbl', 'Alexander Albon'), ('dtotnumprix', '22')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('driver', 'http://www.wikidata.org/entity/Q17319645'), ('driverlbl', 'George Russell'), ('dtotnumprix', '1')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('countrylbl', 'Switzerland'), ('driver', 'http://www.wikidata.org/entity/Q82742'), ('driverlbl', 'Romain Grosjean'), ('dtotnumprix', '1')]
[('country', 'http://www.wikidata.org/entity/Q869'), ('countrylbl', 'Thailand'), ('driver', 'http://www.wikidata.org/entity/Q16149172'), ('driverlbl', 'Alexander Albon'), ('dtotnumprix', '22')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countrylbl', 'Brazil'), ('driver', 'http://www.wikidata.org/entity/Q10490'), ('driverlbl', 'Ayrton Senna'), ('dtotnumprix', '13')]
[('country', 'http://w

13

In [31]:
# try to get number of partecipation for each country of drivers and teams (it is bad query)

queryString = """
SELECT ?country ?countrylbl ?driver ?driverlbl (MAX(?dtotnumprix) as ?maxdnumprix) ?team ?teamlbl (MAX(?ttotnumprix) as ?maxtnumprix)
WHERE {

{
    SELECT ?country ?countrylbl ?team ?teamlbl (SUM(?tnumprix) as ?ttotnumprix)
    WHERE {
        SELECT ?country ?countrylbl ?team ?teamlbl (COUNT(?prix) as ?tnumprix)
        WHERE {

        ?team wdt:P31 wd:Q10497835 ;
            wdt:P17 ?country .

        ?champs wdt:P1923 ?team .

        ?champs wdt:P527 ?prix .

        ?champs <http://schema.org/name> ?clabel .
        ?prix <http://schema.org/name> ?plabel .

        ?country <http://schema.org/name> ?countrylbl .
        ?team <http://schema.org/name> ?teamlbl .

        FILTER (CONTAINS(?clabel, "Formula One"))
        FILTER (CONTAINS(?plabel, "Prix"))

        }
        GROUP BY ?country ?countrylbl ?team ?teamlbl
        ORDER BY ?country
    }
    GROUP BY ?country ?countrylbl ?team ?teamlbl
}
UNION
{
    SELECT ?country ?countrylbl ?driver ?driverlbl (SUM(?dnumprix) as ?dtotnumprix)
    WHERE {
        SELECT ?country ?countrylbl ?driver ?driverlbl (COUNT(?prix) as ?dnumprix)
        WHERE {

        ?driver wdt:P106 wd:Q10841764 ;
                wdt:P1344 ?champs ;
                wdt:P27 ?country .

        ?champs wdt:P527 ?prix .

        ?champs <http://schema.org/name> ?clabel .
        ?prix <http://schema.org/name> ?plabel .

        ?country <http://schema.org/name> ?countrylbl .
        ?driver <http://schema.org/name> ?driverlbl .

        FILTER (CONTAINS(?clabel, "Formula One"))

        }
        GROUP BY ?country ?countrylbl ?driver ?driverlbl
        ORDER BY ?country
    }
    GROUP BY ?country ?countrylbl ?driver ?driverlbl
}

}
GROUP BY ?country ?countrylbl ?driver ?driverlbl ?team ?teamlbl
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('driver', 'http://www.wikidata.org/entity/Q16149172'), ('driverlbl', 'Alexander Albon'), ('maxdnumprix', '22')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('driver', 'http://www.wikidata.org/entity/Q172832'), ('driverlbl', 'David Coulthard'), ('maxdnumprix', '1')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countrylbl', 'Canada'), ('driver', 'http://www.wikidata.org/entity/Q172285'), ('driverlbl', 'Eppie Wietzes'), ('maxdnumprix', '3')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q172722'), ('teamlbl', 'Renault F1 Team'), ('maxtnumprix', '59')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q16300222'), ('teamlbl', 'Haas F1 Team'), ('maxtnumprix', '59')]
[('country', 'http://w

24

In [32]:
# get class of which "Germany" is an instance

queryString = """
SELECT ?class ?classlbl
WHERE {

wd:Q9671 wdt:P27 ?country .
?country wdt:P31 ?class .

?class <http://schema.org/name> ?classlbl .

}
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q3624078'), ('classlbl', 'sovereign state')]
[('class', 'http://www.wikidata.org/entity/Q4209223'), ('classlbl', 'Rechtsstaat')]
[('class', 'http://www.wikidata.org/entity/Q43702'), ('classlbl', 'federation')]
[('class', 'http://www.wikidata.org/entity/Q619610'), ('classlbl', 'social state')]
[('class', 'http://www.wikidata.org/entity/Q63791824'), ('classlbl', 'country bordering the Baltic Sea')]
[('class', 'http://www.wikidata.org/entity/Q7270'), ('classlbl', 'republic')]


6

In [33]:
# get objects from "Sovereign state" node

queryString = """
SELECT ?o ?olbl
WHERE {

wd:Q3624078 ?p ?o .

?o <http://schema.org/name> ?olbl .

}
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q7160365'), ('olbl', 'Category:Sovereign states')]
[('o', 'http://www.wikidata.org/entity/Q6256'), ('olbl', 'country')]
[('o', 'http://www.wikidata.org/entity/Q11750'), ('olbl', 'list of sovereign states')]
[('o', 'http://www.wikidata.org/entity/Q1151405'), ('olbl', 'client state')]
[('o', 'http://www.wikidata.org/entity/Q7275'), ('olbl', 'state')]
[('o', 'http://www.wikidata.org/entity/Q3591847'), ('olbl', 'state in international law')]


6

In [34]:
# get objects from path "Sovereign states"->predicate->"country" node

queryString = """
SELECT ?o ?olbl
WHERE {

wd:Q6256 ?p ?o .

?o <http://schema.org/name> ?olbl .

}
ORDER BY ?olbl
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5641433'), ('olbl', 'Category:Categories by country')]
[('o', 'http://www.wikidata.org/entity/Q4026570'), ('olbl', 'Category:Countries')]
[('o', 'http://www.wikidata.org/entity/P935'), ('olbl', 'Commons gallery')]
[('o', 'http://www.wikidata.org/entity/P901'), ('olbl', 'FIPS 10-4 (countries and regions)')]
[('o', 'http://www.wikidata.org/entity/P2988'), ('olbl', 'GOST 7.67 cyrillic')]
[('o', 'http://www.wikidata.org/entity/P3067'), ('olbl', 'GS1 country code')]
[('o', 'http://www.wikidata.org/entity/P1566'), ('olbl', 'GeoNames ID')]
[('o', 'http://www.wikidata.org/entity/Q183'), ('olbl', 'Germany')]
[('o', 'http://www.wikidata.org/entity/P1125'), ('olbl', 'Gini coefficient')]
[('o', 'http://www.wikidata.org/entity/P3106'), ('olbl', 'Guardian topic ID')]
[('o', 'http://www.wikidata.org/entity/P1081'), ('olbl', 'Human Development Index')]
[('o', 'http://www.wikidata.org/entity/P984'), ('olbl', 'IOC country code')]
[('o', 'http://www.wikidat

81

In [35]:
# get objects from "country" node

queryString = """
SELECT ?o2 ?o2lbl
WHERE {

wd:Q6256 ?p ?o .
?o ?p ?o2 .

?o <http://schema.org/name> ?olbl .
?o2 <http://schema.org/name> ?o2lbl .

FILTER (?olbl = "country")

}
ORDER BY ?olbl
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [36]:
# get list of countries

queryString = """
SELECT DISTINCT ?o ?olbl
WHERE {

?s wdt:P17 ?o .

?o <http://schema.org/name> ?olbl .

}
ORDER BY ?olbl
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1129036'), ('olbl', "'Alawi dynasty")]
[('o', 'http://www.wikidata.org/entity/Q13417403'), ('olbl', '.الجزائر')]
[('o', 'http://www.wikidata.org/entity/Q1084346'), ('olbl', '10792 Ecuador')]
[('o', 'http://www.wikidata.org/entity/Q11185405'), ('olbl', '16th')]
[('o', 'http://www.wikidata.org/entity/Q5708703'), ('olbl', 'AGADU')]
[('o', 'http://www.wikidata.org/entity/Q11972'), ('olbl', 'Aargau')]
[('o', 'http://www.wikidata.org/entity/Q479354'), ('olbl', 'Abazinsky District')]
[('o', 'http://www.wikidata.org/entity/Q12536'), ('olbl', 'Abbasid Caliphate')]
[('o', 'http://www.wikidata.org/entity/Q23334'), ('olbl', 'Abkhazia')]
[('o', 'http://www.wikidata.org/entity/Q3787'), ('olbl', 'Abuja')]
[('o', 'http://www.wikidata.org/entity/Q207353'), ('olbl', 'Acadia')]
[('o', 'http://www.wikidata.org/entity/Q1823'), ('olbl', 'Aceh')]
[('o', 'http://www.wikidata.org/entity/Q2790451'), ('olbl', 'Achaea')]
[('o', 'http://www.wikidata.org/entity/Q24479

3025

In [37]:
# try to get number of partecipation for each country of drivers and teams (it is bad query)

queryString = """
SELECT ?country ?countrylbl ?team ?teamlbl ?ttotprix ?driver ?driverlbl ?dtotprix
WHERE {

?s wdt:P17 ?country .

{
    SELECT ?team ?teamlbl (SUM(?numprix) AS ?ttotprix)
    WHERE {
        SELECT ?team ?teamlbl (COUNT(?prix) AS ?numprix)
        WHERE {
            
            ?team wdt:P31 wd:Q10497835 .

            ?champs wdt:P1923 ?team .

            ?champs wdt:P527 ?prix .
            
            ?champs <http://schema.org/name> ?clabel .
            ?prix <http://schema.org/name> ?plabel .
            ?team <http://schema.org/name> ?teamlbl .

            FILTER (CONTAINS(?clabel, "Formula One"))
            FILTER (CONTAINS(?plabel, "Prix"))
        }
    }
    ORDER BY DESC (?ttotprix)
    LIMIT 1
}

{
    SELECT ?driver ?driverlbl (SUM(?numprix) AS ?dtotprix)
    WHERE {
        SELECT ?driver ?driverlbl (COUNT(?prix) AS ?numprix)
        WHERE {
            
            ?driver wdt:P106 wd:Q10841764 ;
                wdt:P1344 ?champs .

            ?champs wdt:P527 ?prix .

            ?champs <http://schema.org/name> ?clabel .
            ?driver <http://schema.org/name> ?driverlbl .

            FILTER (CONTAINS(?clabel, "Formula One"))
        }
    }
    ORDER BY DESC (?dtotprix)
    LIMIT 1
}

?team wdt:P17 ?country .
?driver wdt:P27 ?country .

}
ORDER BY ?countrylbl
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [38]:
# get partecipating team for each season and related number of prix. Get MAX of tnumprix

queryString = """
SELECT ?country ?countrylbl ?team ?teamlbl ?clabel (MAX(?tnumprix) as ?maxnumprix) 
WHERE {

    SELECT ?country ?countrylbl ?team ?teamlbl ?clabel (COUNT(?prix) as ?tnumprix)
    WHERE {

    ?team wdt:P31 wd:Q10497835 ;
        wdt:P17 ?country .

    ?champs wdt:P1923 ?team .

    ?champs wdt:P527 ?prix .

    ?champs <http://schema.org/name> ?clabel .
    ?prix <http://schema.org/name> ?plabel .

    ?country <http://schema.org/name> ?countrylbl .
    ?team <http://schema.org/name> ?teamlbl .

    FILTER (CONTAINS(?clabel, "Formula One"))
    FILTER (CONTAINS(?plabel, "Prix"))

    }
    ORDER BY ?country ?clabel

}
GROUP BY ?country ?countrylbl ?team ?teamlbl ?clabel

"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q173663'), ('teamlbl', 'Red Bull Racing'), ('clabel', '2017 Formula One World Championship'), ('maxnumprix', '20')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q172722'), ('teamlbl', 'Renault F1 Team'), ('clabel', '2017 Formula One World Championship'), ('maxnumprix', '20')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q172722'), ('teamlbl', 'Renault F1 Team'), ('clabel', '2018 Formula One World Championship'), ('maxnumprix', '18')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('team', 'http://www.wikidata.org/entity/Q138933'), ('teamlbl', 'Force India'), ('clabel', '2018 Formula One World Championship'), ('maxnumprix', '18')]
[('country', 'http:/

33

In [39]:
# try to get number of partecipation for each country of drivers and teams (it is bad query)

queryString = """
SELECT ?dcountry ?dcountrylbl ?team ?teamlbl ?ttotprix ?driver ?driverlbl ?dtotprix
WHERE {

{
    SELECT ?team ?teamlbl (SUM(?numprix) AS ?ttotprix)
    WHERE {
    
        SELECT ?team ?teamlbl (COUNT(?prix) AS ?numprix)
        WHERE {
            
            ?team wdt:P31 wd:Q10497835 .

            ?champs wdt:P1923 ?team .

            ?champs wdt:P527 ?prix .
            
            ?champs <http://schema.org/name> ?clabel .
            ?prix <http://schema.org/name> ?plabel .
            ?team <http://schema.org/name> ?teamlbl .

            FILTER (CONTAINS(?clabel, "Formula One"))
            FILTER (CONTAINS(?plabel, "Prix"))
        }
        GROUP BY ?team ?teamlbl
    }
    GROUP BY ?team ?teamlbl
    ORDER BY DESC (?ttotprix)
    LIMIT 1
}

{
    SELECT ?driver ?driverlbl (SUM(?numprix) AS ?dtotprix)
    WHERE {
    
        SELECT ?driver ?driverlbl (COUNT(?prix) AS ?numprix)
        WHERE {
            
            ?driver wdt:P106 wd:Q10841764 ;
                wdt:P1344 ?champs .

            ?champs wdt:P527 ?prix .

            ?champs <http://schema.org/name> ?clabel .
            ?driver <http://schema.org/name> ?driverlbl .

            FILTER (CONTAINS(?clabel, "Formula One"))
        }
        GROUP BY ?driver ?driverlbl
    }
    GROUP BY ?driver ?driverlbl
    ORDER BY DESC (?dtotprix)
    LIMIT 1
}

?driver wdt:P27 ?dcountry .
?team wdt:P17 ?tcountry .

FILTER (?dcountry = ?tcountry)

}
ORDER BY ?dcountrylbl
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [40]:
# get number of partecipation of teams for each country (second method)

queryString = """
SELECT ?country ?countrylbl (GROUP_CONCAT(?team; separator=" | ") AS ?teams)  (GROUP_CONCAT(?teamlbl; separator=" | ") AS ?teamslbl) ?maxtotprix
WHERE
{
    {
        SELECT ?country (MAX(?numprix) AS ?maxtotprix)
        WHERE 
        {
            {
                SELECT ?country ?team (COUNT(?prix) AS ?numprix)
                WHERE {

                    ?team wdt:P31 wd:Q10497835 ;
                        wdt:P17 ?country .

                    ?champs wdt:P1923 ?team .

                    ?champs wdt:P527 ?prix .

                    ?champs <http://schema.org/name> ?clabel .
                    ?prix <http://schema.org/name> ?plabel .

                    FILTER (CONTAINS(?clabel, "Formula One"))
                    FILTER (CONTAINS(?plabel, "Prix"))
                }
                GROUP BY ?country ?team
            }
        }
        GROUP BY ?country
    }
    {
        {
            SELECT ?country ?team (COUNT(?prix) AS ?numprix)
            WHERE {

                ?team wdt:P31 wd:Q10497835 ;
                    wdt:P17 ?country .

                ?champs wdt:P1923 ?team .

                ?champs wdt:P527 ?prix .

                ?champs <http://schema.org/name> ?clabel .
                ?prix <http://schema.org/name> ?plabel .

                FILTER (CONTAINS(?clabel, "Formula One"))
                FILTER (CONTAINS(?plabel, "Prix"))
            }
            GROUP BY ?country ?team
        }
    }
    
    ?country <http://schema.org/name> ?countrylbl .
    ?team <http://schema.org/name> ?teamlbl .
}
ORDER BY ?maxtotprix
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countrylbl', 'Italy'), ('teams', 'http://www.wikidata.org/entity/Q169898 | http://www.wikidata.org/entity/Q141818'), ('teamslbl', 'Scuderia Ferrari | Scuderia Toro Rosso'), ('maxtotprix', '59')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('countrylbl', 'Switzerland'), ('teams', 'http://www.wikidata.org/entity/Q171335'), ('teamslbl', 'Sauber'), ('maxtotprix', '59')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countrylbl', 'France'), ('teams', 'http://www.wikidata.org/entity/Q172722'), ('teamslbl', 'Renault F1 Team'), ('maxtotprix', '59')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('teams', 'http://www.wikidata.org/entity/Q138933 | http://www.wikidata.org/entity/Q172030 | http://www.wikidata.org/entity/Q173663 | http://www.wikidata.org/entity/Q171337 | http://www.wikidata.org/entity/Q16300222 | http://www.wikidata.org/entity/Q172722 | http://www.wikidata.org/enti

4

In [41]:
# get number of partecipation of drivers for each country (second method)

queryString = """
SELECT ?country ?countrylbl (GROUP_CONCAT(?driver; separator=" | ") AS ?drivers)  (GROUP_CONCAT(?driverlbl; separator=" | ") AS ?driverslbl) ?maxtotprix
WHERE
{
    {
        SELECT ?country (MAX(?numprix) AS ?maxtotprix)
        WHERE 
        {
            {
                SELECT ?country ?driver (COUNT(?prix) AS ?numprix)
                WHERE {

                    ?driver wdt:P106 wd:Q10841764 ;
                        wdt:P1344 ?champs ;
                        wdt:P27 ?country .

                    ?champs wdt:P527 ?prix .

                    ?champs <http://schema.org/name> ?clabel .

                    FILTER (CONTAINS(?clabel, "Formula One"))
                }
                GROUP BY ?country ?driver
            }
        }
        GROUP BY ?country
    }
    {
        {
            SELECT ?country ?driver (COUNT(?prix) AS ?numprix)
            WHERE {

                ?driver wdt:P106 wd:Q10841764 ;
                    wdt:P1344 ?champs ;
                    wdt:P27 ?country .

                ?champs wdt:P527 ?prix .

                ?champs <http://schema.org/name> ?clabel .

                FILTER (CONTAINS(?clabel, "Formula One"))
            }
            GROUP BY ?country ?driver
        }
    }
    
    ?country <http://schema.org/name> ?countrylbl .
    ?driver <http://schema.org/name> ?driverlbl .
}
GROUP BY ?country ?countrylbl ?maxtotprix
ORDER BY ?maxtotprix
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q39'), ('countrylbl', 'Switzerland'), ('drivers', 'http://www.wikidata.org/entity/Q82742'), ('driverslbl', 'Romain Grosjean'), ('maxtotprix', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countrylbl', 'France'), ('drivers', 'http://www.wikidata.org/entity/Q82742'), ('driverslbl', 'Romain Grosjean'), ('maxtotprix', '1')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countrylbl', 'Italy'), ('drivers', 'http://www.wikidata.org/entity/Q7084843'), ('driverslbl', 'Antonio Giovinazzi'), ('maxtotprix', '1')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('countrylbl', 'Finland'), ('drivers', 'http://www.wikidata.org/entity/Q171381'), ('driverslbl', 'Keke Rosberg'), ('maxtotprix', '1')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countrylbl', 'Argentina'), ('drivers', 'http://www.wikidata.org/entity/Q173238'), ('driverslbl', 'Carlos Reutemann'), ('maxtotprix', '1')]
[('country', 'http://www.wikidata.org

10

In [42]:
# GOAL (4.3)
# get the highest number of partecipation of drivers and team for each country

queryString = """

SELECT ?country ?countrylbl ?teams ?teamslbl ?tmaxtotprix ?drivers ?driverslbl ?dmaxtotprix
{

    {
    
        SELECT ?country ?countrylbl (GROUP_CONCAT(?team; separator=" | ") AS ?teams)  (GROUP_CONCAT(?teamlbl; separator=" | ") AS ?teamslbl) ?tmaxtotprix
        WHERE
        {
            {
                SELECT ?country (MAX(?numprix) AS ?tmaxtotprix)
                WHERE 
                {
                    {
                        SELECT ?country ?team (COUNT(?prix) AS ?numprix)
                        WHERE {

                            ?team wdt:P31 wd:Q10497835 ;
                                wdt:P17 ?country .

                            ?champs wdt:P1923 ?team .

                            ?champs wdt:P527 ?prix .

                            ?champs <http://schema.org/name> ?clabel .
                            ?prix <http://schema.org/name> ?plabel .

                            FILTER (CONTAINS(?clabel, "Formula One"))
                            FILTER (CONTAINS(?plabel, "Prix"))
                        }
                        GROUP BY ?country ?team
                    }
                }
                GROUP BY ?country
            }
            {
                {
                    SELECT ?country ?team (COUNT(?prix) AS ?numprix)
                    WHERE {

                        ?team wdt:P31 wd:Q10497835 ;
                            wdt:P17 ?country .

                        ?champs wdt:P1923 ?team .

                        ?champs wdt:P527 ?prix .

                        ?champs <http://schema.org/name> ?clabel .
                        ?prix <http://schema.org/name> ?plabel .

                        FILTER (CONTAINS(?clabel, "Formula One"))
                        FILTER (CONTAINS(?plabel, "Prix"))
                    }
                    GROUP BY ?country ?team
                }
            }

            ?country <http://schema.org/name> ?countrylbl .
            ?team <http://schema.org/name> ?teamlbl .
        }
        GROUP BY ?country ?countrylbl ?tmaxtotprix
        ORDER BY ?tmaxtotprix
    
    }


    {
        SELECT ?country ?countrylbl (GROUP_CONCAT(?driver; separator=" | ") AS ?drivers)  (GROUP_CONCAT(?driverlbl; separator=" | ") AS ?driverslbl) ?dmaxtotprix
        WHERE
        {
            {
                SELECT ?country (MAX(?numprix) AS ?dmaxtotprix)
                WHERE 
                {
                    {
                        SELECT ?country ?driver (COUNT(?prix) AS ?numprix)
                        WHERE {

                            ?driver wdt:P106 wd:Q10841764 ;
                                wdt:P1344 ?champs ;
                                wdt:P27 ?country .

                            ?champs wdt:P527 ?prix .

                            ?champs <http://schema.org/name> ?clabel .

                            FILTER (CONTAINS(?clabel, "Formula One"))
                        }
                        GROUP BY ?country ?driver
                    }
                }
                GROUP BY ?country
            }
            {
                {
                    SELECT ?country ?driver (COUNT(?prix) AS ?numprix)
                    WHERE {

                        ?driver wdt:P106 wd:Q10841764 ;
                            wdt:P1344 ?champs ;
                            wdt:P27 ?country .

                        ?champs wdt:P527 ?prix .

                        ?champs <http://schema.org/name> ?clabel .

                        FILTER (CONTAINS(?clabel, "Formula One"))
                    }
                    GROUP BY ?country ?driver
                }
            }

            ?country <http://schema.org/name> ?countrylbl .
            ?driver <http://schema.org/name> ?driverlbl .
        }
        GROUP BY ?country ?countrylbl ?dmaxtotprix
        ORDER BY ?dmaxtotprix
    }
}
GROUP BY ?country
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countrylbl', 'Italy'), ('teams', 'http://www.wikidata.org/entity/Q169898 | http://www.wikidata.org/entity/Q141818'), ('teamslbl', 'Scuderia Ferrari | Scuderia Toro Rosso'), ('tmaxtotprix', '59'), ('drivers', 'http://www.wikidata.org/entity/Q7084843'), ('driverslbl', 'Antonio Giovinazzi'), ('dmaxtotprix', '1')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('countrylbl', 'Switzerland'), ('teams', 'http://www.wikidata.org/entity/Q171335'), ('teamslbl', 'Sauber'), ('tmaxtotprix', '59'), ('drivers', 'http://www.wikidata.org/entity/Q82742'), ('driverslbl', 'Romain Grosjean'), ('dmaxtotprix', '1')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countrylbl', 'United Kingdom'), ('teams', 'http://www.wikidata.org/entity/Q138933 | http://www.wikidata.org/entity/Q172030 | http://www.wikidata.org/entity/Q173663 | http://www.wikidata.org/entity/Q171337 | http://www.wikidata.org/entity/Q16300222 | http://www.wikidata.org/ent

4